In [14]:
import hyper_param as hp
import numpy as np
import operator
import collections
import random
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = 20
#logging.info("")

In [26]:
#Define constants
tau = 10 #price history window
t_0 = 1000 #start point in history
num_dyna_updates = 25
ntrials = 10000
epsilon = 0.2
horizon = 100
gamma = 0.9

In [16]:
price = hp.get_price_history()[t_0:]

In [17]:
price

array([[  8.03],
       [  7.32],
       [  7.3 ],
       ..., 
       [ 51.93],
       [ 52.32],
       [ 51.48]])

In [18]:
'''
Returns r, sp
'''
def sim(t, s, a):
    x = [t+i for i in range(tau)]
    y = price[t-tau:t]
    slope, intercept = np.polyfit(x, y, 1)
    r = 0
    ncoin = s[1]
    if ncoin <= -50 or ncoin >= 50:
        return ((-price[t]*ncoin)[0], (hp.get_interval_enum(slope[0]), 0))
    else:
        if a == -1: # buy
            r = 0-price[t][0]
            ncoin += 1
        elif a == 1: # sell
            r = price[t][0]
            ncoin -= 1
        elif a == 0:
            reward = 0
        else:
            #throw expection
            print("Invalid action")
        sp = (hp.get_interval_enum(slope[0]), ncoin)
        return r, sp

In [19]:
'''
initialize N's, p, Q
'''
N_sasp = {} # key = N_sasp[(s, a)][sp] #value = transition count
N_sa = collections.defaultdict(int) #key = (s, a) #value = occurence count
p = collections.defaultdict(int) #key = (s, a) #value = sum of rewards
Q = {} # Q[s][a] = q value of taking action a in state s

In [20]:
'''
Returns chosen action
'''
def choose_action_egreedy(s, Q):
    if s in Q:
        Q_value = Q[s]
        prob = random.uniform(0, 1)
        if prob <= 1 - epsilon:  #exploit
            action = max(Q_value, key=Q_value.get)
            return action
        else:  #explore
            action = random.choice([-1, 0, 1])
            if action in Q_value:
                return action
            else:
                return action
    else:
        action = random.choice([-1, 0, 1])
        return action

In [21]:
def get_new_random_state_action(Q, updated):
    for i in range(5):
        rand_s = random.choice(Q.keys())
        rand_a = random.choice(Q[rand_s].keys())
        if (rand_s, rand_a) not in updated:
            return rand_s, rand_a
    return None, None

In [22]:
def dyna(Q, s, a, N_sa, N_sasp, p):
    R_sa = p[(s, a)]/ N_sa[(s, a)]
    
    summation = 0
    for sp in N_sasp[(s, a)]:
        T_sasp = N_sasp[(s, a)][sp]/N_sa[(s, a)]
        #compute max_a'[Q(s',a')]
        if sp not in Q:
            continue #Q = 0
        max_Q = max(Q[sp].iteritems(), key=operator.itemgetter(1))[1]
        summation += T_sasp*max_Q
    
    if s not in Q:
        Q[s] = {}
    if a not in Q[s]:
        Q[s][a] = 0
        
    Q[s][a] = R_sa + gamma*summation

In [23]:
def mle_based_rl(t, s_0):
    s = s_0   #initial state (ncoins, slope_interval)
    curr_horizon = t + horizon
    while (t < curr_horizon):
        
        a = choose_action_egreedy(s, Q)
        r, sp = sim(t, s, a)

        #update N counts
        if (s , a) not in N_sa:
            N_sasp[(s, a)] = collections.defaultdict(int)
            
        N_sa[(s, a)] += 1
        N_sasp[(s, a)][sp] += 1
        
        #update p
        p[(s, a)] += r
        
        dyna(Q, s, a, N_sa, N_sasp, p)
        
        #update Q using Dyna strategy
        updated = set((s,a))
        for i in range(num_dyna_updates):
            s_rand, a_rand = get_new_random_state_action(Q, updated)
            if s_rand is None:   # nothing to update
                break
            dyna(Q, s_rand, a_rand, N_sa, N_sasp, p)
            updated.add((s_rand, a_rand))
        
        s = sp
        t += 1

In [ ]:
for i in range(ntrials):
    t = random.randint(t_0, len(price)-100)
    s_0 = (0, 0)
    if t > t_0:
        window = min(tau, t-t_0)
        x = [t+i for i in range(window)]
        y = price[t-window:t]
        slope, intercept = np.polyfit(x, y, 1)
        s_0 = (hp.get_interval_enum(slope[0]), 0)
    
    logging.info("MLE(t=%d, s_0=[%d, %d])", t, s_0[0], s_0[1])
    mle_based_rl(t, s_0)
    
print(Q)

INFO : MLE(t=1105, s_0=[10, 0])
INFO : MLE(t=1086, s_0=[20, 0])
INFO : MLE(t=1193, s_0=[8, 0])
INFO : MLE(t=1310, s_0=[0, 0])
INFO : MLE(t=1472, s_0=[0, 0])
INFO : MLE(t=1476, s_0=[16, 0])
INFO : MLE(t=1100, s_0=[20, 0])
INFO : MLE(t=1426, s_0=[0, 0])
INFO : MLE(t=1021, s_0=[13, 0])
INFO : MLE(t=1083, s_0=[20, 0])
INFO : MLE(t=1154, s_0=[20, 0])
INFO : MLE(t=1462, s_0=[16, 0])
INFO : MLE(t=1154, s_0=[20, 0])
INFO : MLE(t=1255, s_0=[20, 0])
INFO : MLE(t=1045, s_0=[20, 0])
INFO : MLE(t=1103, s_0=[20, 0])
INFO : MLE(t=1466, s_0=[20, 0])
INFO : MLE(t=1243, s_0=[20, 0])
INFO : MLE(t=1018, s_0=[12, 0])
INFO : MLE(t=1093, s_0=[2, 0])
INFO : MLE(t=1220, s_0=[10, 0])
INFO : MLE(t=1265, s_0=[20, 0])
INFO : MLE(t=1291, s_0=[0, 0])
INFO : MLE(t=1227, s_0=[20, 0])
INFO : MLE(t=1239, s_0=[20, 0])
INFO : MLE(t=1368, s_0=[0, 0])
INFO : MLE(t=1289, s_0=[1, 0])
INFO : MLE(t=1373, s_0=[1, 0])
INFO : MLE(t=1169, s_0=[0, 0])
INFO : MLE(t=1359, s_0=[8, 0])
INFO : MLE(t=1066, s_0=[8, 0])
INFO : MLE(t=1032, s

INFO : MLE(t=1036, s_0=[15, 0])
INFO : MLE(t=1307, s_0=[0, 0])
INFO : MLE(t=1135, s_0=[17, 0])
INFO : MLE(t=1241, s_0=[20, 0])
INFO : MLE(t=1014, s_0=[11, 0])
INFO : MLE(t=1146, s_0=[12, 0])
INFO : MLE(t=1259, s_0=[20, 0])
INFO : MLE(t=1454, s_0=[0, 0])
INFO : MLE(t=1270, s_0=[7, 0])
INFO : MLE(t=1174, s_0=[0, 0])
INFO : MLE(t=1387, s_0=[20, 0])
INFO : MLE(t=1082, s_0=[20, 0])
INFO : MLE(t=1435, s_0=[19, 0])
INFO : MLE(t=1172, s_0=[0, 0])
INFO : MLE(t=1298, s_0=[0, 0])
INFO : MLE(t=1174, s_0=[0, 0])
INFO : MLE(t=1387, s_0=[20, 0])
INFO : MLE(t=1079, s_0=[11, 0])
INFO : MLE(t=1152, s_0=[20, 0])
INFO : MLE(t=1198, s_0=[14, 0])
INFO : MLE(t=1024, s_0=[10, 0])
INFO : MLE(t=1371, s_0=[0, 0])
INFO : MLE(t=1239, s_0=[20, 0])
INFO : MLE(t=1115, s_0=[9, 0])
INFO : MLE(t=1363, s_0=[9, 0])
INFO : MLE(t=1249, s_0=[20, 0])
INFO : MLE(t=1165, s_0=[1, 0])
INFO : MLE(t=1107, s_0=[2, 0])
INFO : MLE(t=1468, s_0=[11, 0])
INFO : MLE(t=1120, s_0=[10, 0])
INFO : MLE(t=1331, s_0=[20, 0])
INFO : MLE(t=1136, s

INFO : MLE(t=1434, s_0=[19, 0])
INFO : MLE(t=1371, s_0=[0, 0])
INFO : MLE(t=1299, s_0=[0, 0])
INFO : MLE(t=1390, s_0=[20, 0])
INFO : MLE(t=1177, s_0=[1, 0])
INFO : MLE(t=1059, s_0=[13, 0])
INFO : MLE(t=1287, s_0=[20, 0])
INFO : MLE(t=1302, s_0=[0, 0])
INFO : MLE(t=1283, s_0=[1, 0])
INFO : MLE(t=1206, s_0=[15, 0])
INFO : MLE(t=1179, s_0=[15, 0])
INFO : MLE(t=1459, s_0=[0, 0])
INFO : MLE(t=1155, s_0=[20, 0])
INFO : MLE(t=1000, s_0=[0, 0])
INFO : MLE(t=1262, s_0=[20, 0])
INFO : MLE(t=1064, s_0=[3, 0])
INFO : MLE(t=1053, s_0=[8, 0])
INFO : MLE(t=1471, s_0=[0, 0])
INFO : MLE(t=1085, s_0=[20, 0])
INFO : MLE(t=1018, s_0=[12, 0])
INFO : MLE(t=1201, s_0=[20, 0])
INFO : MLE(t=1353, s_0=[0, 0])
INFO : MLE(t=1168, s_0=[0, 0])
INFO : MLE(t=1288, s_0=[20, 0])
INFO : MLE(t=1239, s_0=[20, 0])
INFO : MLE(t=1156, s_0=[20, 0])
INFO : MLE(t=1138, s_0=[12, 0])
INFO : MLE(t=1156, s_0=[20, 0])
INFO : MLE(t=1201, s_0=[20, 0])
INFO : MLE(t=1276, s_0=[0, 0])
INFO : MLE(t=1037, s_0=[18, 0])
INFO : MLE(t=1265, s_

INFO : MLE(t=1374, s_0=[5, 0])
INFO : MLE(t=1082, s_0=[20, 0])
INFO : MLE(t=1057, s_0=[19, 0])
INFO : MLE(t=1249, s_0=[20, 0])
INFO : MLE(t=1046, s_0=[15, 0])
INFO : MLE(t=1310, s_0=[0, 0])
INFO : MLE(t=1208, s_0=[4, 0])
INFO : MLE(t=1354, s_0=[0, 0])
INFO : MLE(t=1111, s_0=[0, 0])
INFO : MLE(t=1090, s_0=[4, 0])
INFO : MLE(t=1212, s_0=[4, 0])
INFO : MLE(t=1245, s_0=[20, 0])
INFO : MLE(t=1446, s_0=[0, 0])
INFO : MLE(t=1287, s_0=[20, 0])
INFO : MLE(t=1377, s_0=[6, 0])
INFO : MLE(t=1106, s_0=[5, 0])
INFO : MLE(t=1024, s_0=[10, 0])
INFO : MLE(t=1195, s_0=[7, 0])
INFO : MLE(t=1399, s_0=[7, 0])
INFO : MLE(t=1406, s_0=[20, 0])
INFO : MLE(t=1121, s_0=[9, 0])
INFO : MLE(t=1028, s_0=[17, 0])
INFO : MLE(t=1348, s_0=[0, 0])
INFO : MLE(t=1341, s_0=[8, 0])
INFO : MLE(t=1396, s_0=[11, 0])
INFO : MLE(t=1018, s_0=[12, 0])
INFO : MLE(t=1027, s_0=[15, 0])
INFO : MLE(t=1422, s_0=[0, 0])
INFO : MLE(t=1328, s_0=[20, 0])
INFO : MLE(t=1044, s_0=[20, 0])
INFO : MLE(t=1283, s_0=[1, 0])
INFO : MLE(t=1098, s_0=[1

INFO : MLE(t=1142, s_0=[5, 0])
INFO : MLE(t=1320, s_0=[20, 0])
INFO : MLE(t=1347, s_0=[0, 0])
INFO : MLE(t=1487, s_0=[11, 0])
INFO : MLE(t=1077, s_0=[13, 0])
INFO : MLE(t=1408, s_0=[12, 0])
INFO : MLE(t=1173, s_0=[0, 0])
INFO : MLE(t=1229, s_0=[18, 0])
INFO : MLE(t=1053, s_0=[8, 0])
INFO : MLE(t=1102, s_0=[20, 0])
INFO : MLE(t=1074, s_0=[12, 0])
INFO : MLE(t=1379, s_0=[1, 0])
INFO : MLE(t=1137, s_0=[15, 0])
INFO : MLE(t=1168, s_0=[0, 0])
INFO : MLE(t=1349, s_0=[0, 0])
INFO : MLE(t=1415, s_0=[0, 0])
INFO : MLE(t=1276, s_0=[0, 0])
INFO : MLE(t=1406, s_0=[20, 0])
INFO : MLE(t=1164, s_0=[12, 0])
INFO : MLE(t=1358, s_0=[1, 0])
INFO : MLE(t=1270, s_0=[7, 0])
INFO : MLE(t=1356, s_0=[0, 0])
INFO : MLE(t=1385, s_0=[20, 0])
INFO : MLE(t=1358, s_0=[1, 0])
INFO : MLE(t=1007, s_0=[16, 0])
INFO : MLE(t=1208, s_0=[4, 0])
INFO : MLE(t=1006, s_0=[17, 0])
INFO : MLE(t=1388, s_0=[20, 0])
INFO : MLE(t=1297, s_0=[0, 0])
INFO : MLE(t=1363, s_0=[9, 0])
INFO : MLE(t=1161, s_0=[20, 0])
INFO : MLE(t=1077, s_0=[

...

INFO : MLE(t=1488, s_0=[12, 0])
INFO : MLE(t=1094, s_0=[1, 0])
INFO : MLE(t=1375, s_0=[11, 0])
INFO : MLE(t=1046, s_0=[15, 0])
INFO : MLE(t=1129, s_0=[12, 0])
INFO : MLE(t=1375, s_0=[11, 0])
INFO : MLE(t=1248, s_0=[20, 0])
INFO : MLE(t=1288, s_0=[20, 0])
INFO : MLE(t=1101, s_0=[20, 0])
INFO : MLE(t=1204, s_0=[20, 0])
INFO : MLE(t=1012, s_0=[14, 0])
INFO : MLE(t=1066, s_0=[8, 0])
INFO : MLE(t=1085, s_0=[20, 0])
INFO : MLE(t=1046, s_0=[15, 0])
INFO : MLE(t=1331, s_0=[20, 0])
INFO : MLE(t=1206, s_0=[15, 0])
INFO : MLE(t=1167, s_0=[0, 0])
INFO : MLE(t=1061, s_0=[7, 0])
INFO : MLE(t=1232, s_0=[15, 0])
INFO : MLE(t=1130, s_0=[15, 0])
INFO : MLE(t=1461, s_0=[12, 0])
INFO : MLE(t=1217, s_0=[10, 0])
INFO : MLE(t=1090, s_0=[4, 0])
INFO : MLE(t=1344, s_0=[0, 0])
INFO : MLE(t=1093, s_0=[2, 0])
INFO : MLE(t=1267, s_0=[20, 0])
INFO : MLE(t=1290, s_0=[0, 0])
INFO : MLE(t=1231, s_0=[17, 0])
INFO : MLE(t=1036, s_0=[15, 0])
INFO : MLE(t=1322, s_0=[20, 0])
INFO : MLE(t=1487, s_0=[11, 0])
INFO : MLE(t=121

INFO : MLE(t=1002, s_0=[20, 0])
INFO : MLE(t=1321, s_0=[20, 0])
INFO : MLE(t=1203, s_0=[20, 0])
INFO : MLE(t=1073, s_0=[14, 0])
INFO : MLE(t=1276, s_0=[0, 0])
INFO : MLE(t=1460, s_0=[6, 0])
INFO : MLE(t=1291, s_0=[0, 0])
INFO : MLE(t=1178, s_0=[8, 0])
INFO : MLE(t=1387, s_0=[20, 0])
INFO : MLE(t=1267, s_0=[20, 0])
INFO : MLE(t=1358, s_0=[1, 0])
INFO : MLE(t=1157, s_0=[20, 0])
INFO : MLE(t=1139, s_0=[10, 0])
INFO : MLE(t=1405, s_0=[20, 0])
INFO : MLE(t=1052, s_0=[5, 0])
INFO : MLE(t=1087, s_0=[20, 0])
INFO : MLE(t=1162, s_0=[20, 0])
INFO : MLE(t=1004, s_0=[18, 0])
INFO : MLE(t=1203, s_0=[20, 0])
INFO : MLE(t=1477, s_0=[20, 0])
INFO : MLE(t=1120, s_0=[10, 0])
INFO : MLE(t=1366, s_0=[0, 0])
INFO : MLE(t=1091, s_0=[2, 0])
INFO : MLE(t=1393, s_0=[20, 0])
INFO : MLE(t=1368, s_0=[0, 0])
INFO : MLE(t=1099, s_0=[20, 0])
INFO : MLE(t=1440, s_0=[0, 0])
INFO : MLE(t=1093, s_0=[2, 0])
INFO : MLE(t=1409, s_0=[0, 0])
INFO : MLE(t=1134, s_0=[17, 0])
INFO : MLE(t=1259, s_0=[20, 0])
INFO : MLE(t=1349, s

INFO : MLE(t=1268, s_0=[20, 0])
INFO : MLE(t=1042, s_0=[20, 0])
INFO : MLE(t=1397, s_0=[10, 0])
INFO : MLE(t=1045, s_0=[20, 0])
INFO : MLE(t=1004, s_0=[18, 0])
INFO : MLE(t=1167, s_0=[0, 0])
INFO : MLE(t=1009, s_0=[17, 0])
INFO : MLE(t=1242, s_0=[20, 0])
INFO : MLE(t=1294, s_0=[0, 0])
INFO : MLE(t=1336, s_0=[8, 0])
INFO : MLE(t=1026, s_0=[14, 0])
INFO : MLE(t=1322, s_0=[20, 0])
INFO : MLE(t=1339, s_0=[17, 0])
INFO : MLE(t=1193, s_0=[8, 0])
INFO : MLE(t=1343, s_0=[0, 0])
INFO : MLE(t=1226, s_0=[20, 0])
INFO : MLE(t=1447, s_0=[0, 0])
INFO : MLE(t=1488, s_0=[12, 0])
INFO : MLE(t=1247, s_0=[20, 0])
INFO : MLE(t=1016, s_0=[11, 0])
INFO : MLE(t=1130, s_0=[15, 0])
INFO : MLE(t=1355, s_0=[0, 0])
INFO : MLE(t=1291, s_0=[0, 0])
INFO : MLE(t=1334, s_0=[0, 0])
INFO : MLE(t=1176, s_0=[0, 0])
INFO : MLE(t=1158, s_0=[20, 0])
INFO : MLE(t=1269, s_0=[20, 0])
INFO : MLE(t=1279, s_0=[0, 0])
INFO : MLE(t=1490, s_0=[7, 0])
INFO : MLE(t=1484, s_0=[9, 0])
INFO : MLE(t=1147, s_0=[13, 0])
INFO : MLE(t=1051, s_

INFO : MLE(t=1033, s_0=[14, 0])
INFO : MLE(t=1473, s_0=[0, 0])
INFO : MLE(t=1432, s_0=[17, 0])
INFO : MLE(t=1370, s_0=[0, 0])
INFO : MLE(t=1073, s_0=[14, 0])
INFO : MLE(t=1034, s_0=[13, 0])
INFO : MLE(t=1472, s_0=[0, 0])
INFO : MLE(t=1314, s_0=[0, 0])
INFO : MLE(t=1013, s_0=[13, 0])
INFO : MLE(t=1480, s_0=[20, 0])
INFO : MLE(t=1302, s_0=[0, 0])
INFO : MLE(t=1455, s_0=[0, 0])
INFO : MLE(t=1372, s_0=[0, 0])
INFO : MLE(t=1402, s_0=[20, 0])
INFO : MLE(t=1444, s_0=[0, 0])
INFO : MLE(t=1453, s_0=[0, 0])
INFO : MLE(t=1143, s_0=[7, 0])
INFO : MLE(t=1446, s_0=[0, 0])
INFO : MLE(t=1139, s_0=[10, 0])
INFO : MLE(t=1224, s_0=[12, 0])
INFO : MLE(t=1481, s_0=[18, 0])
INFO : MLE(t=1301, s_0=[0, 0])
INFO : MLE(t=1082, s_0=[20, 0])
INFO : MLE(t=1248, s_0=[20, 0])
INFO : MLE(t=1206, s_0=[15, 0])
INFO : MLE(t=1459, s_0=[0, 0])
INFO : MLE(t=1012, s_0=[14, 0])
INFO : MLE(t=1044, s_0=[20, 0])
INFO : MLE(t=1465, s_0=[19, 0])
INFO : MLE(t=1385, s_0=[20, 0])
INFO : MLE(t=1434, s_0=[19, 0])
INFO : MLE(t=1263, s_

INFO : MLE(t=1351, s_0=[0, 0])
INFO : MLE(t=1000, s_0=[0, 0])
INFO : MLE(t=1055, s_0=[9, 0])
INFO : MLE(t=1183, s_0=[11, 0])
INFO : MLE(t=1145, s_0=[10, 0])
INFO : MLE(t=1162, s_0=[20, 0])
INFO : MLE(t=1007, s_0=[16, 0])
INFO : MLE(t=1289, s_0=[1, 0])
INFO : MLE(t=1347, s_0=[0, 0])
INFO : MLE(t=1297, s_0=[0, 0])
INFO : MLE(t=1488, s_0=[12, 0])
INFO : MLE(t=1079, s_0=[11, 0])
INFO : MLE(t=1161, s_0=[20, 0])
INFO : MLE(t=1009, s_0=[17, 0])
INFO : MLE(t=1020, s_0=[12, 0])
INFO : MLE(t=1173, s_0=[0, 0])
INFO : MLE(t=1341, s_0=[8, 0])
INFO : MLE(t=1267, s_0=[20, 0])
INFO : MLE(t=1261, s_0=[20, 0])
INFO : MLE(t=1407, s_0=[20, 0])
INFO : MLE(t=1147, s_0=[13, 0])
INFO : MLE(t=1153, s_0=[20, 0])
INFO : MLE(t=1174, s_0=[0, 0])
INFO : MLE(t=1144, s_0=[9, 0])
INFO : MLE(t=1218, s_0=[10, 0])
INFO : MLE(t=1491, s_0=[3, 0])
INFO : MLE(t=1055, s_0=[9, 0])
INFO : MLE(t=1270, s_0=[7, 0])
INFO : MLE(t=1042, s_0=[20, 0])
INFO : MLE(t=1277, s_0=[0, 0])
INFO : MLE(t=1301, s_0=[0, 0])
INFO : MLE(t=1040, s_0=

In [ ]:
def generate_mle_policy(Q, t, price):
    policy = []
    cur_num_coins = 0
    for time in range(t, t+hyper_param.get_policy_length()):
        x = [t+i for i in range(10)]
        y = price[t-10:t]
        slope, intercept = np.polyfit(x, y, 1)
        interval_group = get_interval_enum(slope[0])
        if (interval_group, cur_num_coins) in Q: 
            Q_value = Q[(interval_group, cur_num_coins)]
            action = max(Q_value, key=Q_value.get)
            if Q_value[action] < 0: # meaningless maximum
                action = random.choice([-1, 0, 1])
        else:
            action = random.choice([-1, 0, 1])
        if action == -1:
            cur_num_coins += 1
        elif action == 1:
            cur_num_coins -= 1
        else:
            pass
        policy.append(action)
    return policy